In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import datetime, pytz
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [ ]:
# Load NVIDIA stock data from Yahoo Finance
nvidia = yf.download('NVDA', start='2018-01-01', end='2024-01-01', interval='1d')
nvidia.head()


In [ ]:
# Define technical indicator functions
def moving_average(df, window):
    return df['Close'].rolling(window=window).mean()

def exponential_moving_average(df, window):
    return df['Close'].ewm(span=window, adjust=False).mean()

def momentum(df, window):
    return df['Close'].diff(window)

def rsi(df, window):
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Calculate technical indicators
nvidia['SMA10'] = moving_average(nvidia, 10)
nvidia['SMA60'] = moving_average(nvidia, 60)
nvidia['EMA10'] = exponential_moving_average(nvidia, 10)
nvidia['Momentum'] = momentum(nvidia, 10)
nvidia['RSI'] = rsi(nvidia, 14)

# Drop NaN values
nvidia.dropna(inplace=True)
nvidia.head()


In [ ]:
# Generate signals
nvidia['signal'] = 0
nvidia['signal'][10:] = np.where(nvidia['SMA10'][10:] > nvidia['SMA60'][10:], 1, 0)
nvidia['positions'] = nvidia['signal'].diff()

# Visualize signals
plt.figure(figsize=(12, 6))
plt.plot(nvidia['Close'], label='Close Price')
plt.plot(nvidia['SMA10'], label='SMA10')
plt.plot(nvidia['SMA60'], label='SMA60')
plt.plot(nvidia[nvidia['positions'] == 1].index, nvidia['SMA10'][nvidia['positions'] == 1], '^', markersize=10, color='g', label='Buy Signal')
plt.plot(nvidia[nvidia['positions'] == -1].index, nvidia['SMA10'][nvidia['positions'] == -1], 'v', markersize=10, color='r', label='Sell Signal')
plt.title('NVIDIA Close Price and Buy/Sell Signals')
plt.legend()
plt.show()


In [ ]:
# Split the data into training and testing sets
train_size = int(len(nvidia) * 0.8)
train_df = nvidia[:train_size]
test_df = nvidia[train_size:]

# Features and target variable
X_train = train_df[['SMA10', 'SMA60', 'EMA10', 'Momentum', 'RSI']]
y_train = train_df['signal']
X_test = test_df[['SMA10', 'SMA60', 'EMA10', 'Momentum', 'RSI']]
y_test = test_df['signal']


In [ ]:
# Train RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=25)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Train XGBClassifier
xgb_model = XGBClassifier(n_estimators=25, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

# Add predictions to the test dataframe
test_df['rf_signal'] = rf_predictions
test_df['xgb_signal'] = xgb_predictions


In [ ]:
# Function to calculate returns and Sharpe ratio
def calculate_performance(df, signal_col):
    df['returns'] = df['Close'].pct_change()
    df['strategy_returns'] = df[signal_col].shift(1) * df['returns']
    cumulative_returns = df['strategy_returns'].cumsum()

    sharpe_ratio = (df['strategy_returns'].mean() / df['strategy_returns'].std()) * np.sqrt(252)

    annualized_return = df['strategy_returns'].mean() * 252
    annualized_volatility = df['strategy_returns'].std() * np.sqrt(252)
    annualized_sharpe_ratio = annualized_return / annualized_volatility

    return cumulative_returns, sharpe_ratio, annualized_sharpe_ratio

# Calculate performance for RandomForest strategy
rf_cumulative_returns, rf_sharpe_ratio, rf_annualized_sharpe_ratio = calculate_performance(test_df, 'rf_signal')

# Calculate performance for XGBoost strategy
xgb_cumulative_returns, xgb_sharpe_ratio, xgb_annualized_sharpe_ratio = calculate_performance(test_df, 'xgb_signal')

# Print performance metrics
print(f"RandomForest Sharpe Ratio: {rf_sharpe_ratio:.2f}")
print(f"RandomForest Annualized Sharpe Ratio: {rf_annualized_sharpe_ratio:.2f}")
print(f"XGBoost Sharpe Ratio: {xgb_sharpe_ratio:.2f}")
print(f"XGBoost Annualized Sharpe Ratio: {xgb_annualized_sharpe_ratio:.2f}")

# Plot cumulative returns
plt.figure(figsize=(12, 6))
plt.plot(rf_cumulative_returns, label='RandomForest Cumulative Returns')
plt.plot(xgb_cumulative_returns, label='XGBoost Cumulative Returns')
plt.title('Cumulative Returns of Trading Strategies')
plt.legend()
plt.show()
